In [1]:
import requests
try:
    r = requests.get('https://storal.fr/products/kissimy', timeout=10)
    print(r.status_code)
    print(r.text[:500])
except Exception as e:
    print(e)

200
<!DOCTYPE html><html lang="fr"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="preload" href="/_next/static/media/4cf2300e9c8272f7-s.p.woff2" as="font" crossorigin="" type="font/woff2"/><link rel="preload" href="/_next/static/media/93f479601ee12b01-s.p.woff2" as="font" crossorigin="" type="font/woff2"/><link rel="stylesheet" href="/_next/static/css/6b0b5caa044fc863.css" data-precedence="next"/><link rel="preload" as="script" fetchPrior


In [2]:
# Vérifier les fichiers CSS et JS demandés par le HTML
import re
css_files = re.findall(r'href="(/_next/static/css/[^"]+)"', r.text)
js_files = re.findall(r'src="(/_next/static/chunks/[^"]+\.js)"', r.text)

print("CSS files referenced:")
for css in css_files[:3]:
    print(f"  {css}")
    
print("\nJS files referenced:")
for js in js_files[:3]:
    print(f"  {js}")

# Tester si ces fichiers existent
print("\nTesting CSS file accessibility:")
if css_files:
    css_url = f"https://storal.fr{css_files[0]}"
    css_resp = requests.get(css_url, timeout=5)
    print(f"  {css_url} -> {css_resp.status_code}")

CSS files referenced:
  /_next/static/css/6b0b5caa044fc863.css

JS files referenced:
  /_next/static/chunks/4bd1b696-e5d7c65570c947b7.js
  /_next/static/chunks/3794-0fa92e90b939f74c.js
  /_next/static/chunks/main-app-c21d74f9618a768c.js

Testing CSS file accessibility:
  https://storal.fr/_next/static/css/6b0b5caa044fc863.css -> 200


In [3]:
# Tester la page d'accueil
print("=== PAGE D'ACCUEIL ===")
home = requests.get('https://storal.fr/', timeout=10)
print(f"Status: {home.status_code}")

# Chercher les CSS
css_home = re.findall(r'href="(/_next/static/css/[^"]+)"', home.text)
print(f"CSS files: {css_home}")

# Tester si le CSS existe
if css_home:
    css_url = f"https://storal.fr{css_home[0]}"
    css_test = requests.get(css_url, timeout=5)
    print(f"CSS accessibility: {css_url} -> {css_test.status_code}")
    
# Chercher les erreurs JS
if 'error' in home.text.lower():
    print("⚠️ 'error' found in HTML")
    
print("\n=== PAGE KISSIMY ===")
# Tester les JS files pour kissimy
js_test = requests.get('https://storal.fr/_next/static/chunks/3794-0fa92e90b939f74c.js', timeout=5)
print(f"JS chunk 3794: {js_test.status_code}")

js_test2 = requests.get('https://storal.fr/_next/static/chunks/aaea2bcf-d7434765e0b14d18.js', timeout=5)
print(f"JS chunk aaea2bcf: {js_test2.status_code}")

=== PAGE D'ACCUEIL ===
Status: 200
CSS files: ['/_next/static/css/c98d9c18f51f4126.css']
CSS accessibility: https://storal.fr/_next/static/css/c98d9c18f51f4126.css -> 404
⚠️ 'error' found in HTML

=== PAGE KISSIMY ===
JS chunk 3794: 404
JS chunk aaea2bcf: 404


In [4]:
# Attendre 10 secondes et retester
import time
time.sleep(10)

print("=== RETEST APRÈS REDÉMARRAGE PM2 ===")
home = requests.get('https://storal.fr/', timeout=10)
print(f"Page d'accueil: {home.status_code}")
css_home = re.findall(r'href="(/_next/static/css/[^"]+)"', home.text)
print(f"CSS demandé: {css_home}")

if css_home:
    css_url = f"https://storal.fr{css_home[0]}"
    css_test = requests.get(css_url, timeout=5)
    print(f"CSS test: {css_test.status_code}")
    if css_test.status_code == 200:
        print("✅ CSS OK!")
    else:
        print("❌ CSS toujours 404")

=== RETEST APRÈS REDÉMARRAGE PM2 ===
Page d'accueil: 200
CSS demandé: ['/_next/static/css/c98d9c18f51f4126.css']
CSS test: 404
❌ CSS toujours 404


In [5]:
# Monitorer le serveur toutes les 15 secondes
for i in range(8):  # 2 minutes max
    time.sleep(15)
    try:
        r = requests.get('https://storal.fr/', timeout=5)
        print(f"Tentative {i+1}: Status {r.status_code}")
        if r.status_code == 200:
            css = re.findall(r'href="(/_next/static/css/[^"]+)"', r.text)
            if css:
                css_test = requests.get(f"https://storal.fr{css[0]}", timeout=5)
                print(f"  CSS: {css[0]} -> {css_test.status_code}")
                if css_test.status_code == 200:
                    print("✅ SERVEUR OK!")
                    break
    except Exception as e:
        print(f"Tentative {i+1}: {e}")

Tentative 1: Status 200
  CSS: /_next/static/css/c98d9c18f51f4126.css -> 404
Tentative 2: Status 200
  CSS: /_next/static/css/c98d9c18f51f4126.css -> 404
Tentative 3: Status 200
  CSS: /_next/static/css/c98d9c18f51f4126.css -> 404
Tentative 4: Status 200
  CSS: /_next/static/css/c98d9c18f51f4126.css -> 404
Tentative 5: Status 200
  CSS: /_next/static/css/c98d9c18f51f4126.css -> 404
Tentative 6: Status 200
  CSS: /_next/static/css/c98d9c18f51f4126.css -> 404
Tentative 7: Status 200
  CSS: /_next/static/css/c98d9c18f51f4126.css -> 404
Tentative 8: Status 200
  CSS: /_next/static/css/c98d9c18f51f4126.css -> 404


In [6]:
# Attendre le rebuild (3 minutes) puis retester
print("Attente du rebuild complet...")
time.sleep(180)

print("\n=== TEST FINAL ===")
home = requests.get('https://storal.fr/', timeout=10)
print(f"Page d'accueil: {home.status_code}")

css_files = re.findall(r'href="(/_next/static/css/[^"]+)"', home.text)
print(f"CSS demandés: {css_files}")

if css_files:
    for css in css_files:
        css_url = f"https://storal.fr{css}"
        css_test = requests.get(css_url, timeout=5)
        print(f"  {css} -> {css_test.status_code}")
        if css_test.status_code == 200:
            print("    ✅ OK")
        else:
            print("    ❌ 404")
            
print("\n=== TEST KISSIMY ===")
kissimy = requests.get('https://storal.fr/products/kissimy', timeout=10)
print(f"Page Kissimy: {kissimy.status_code}")
js_files = re.findall(r'src="(/_next/static/chunks/[^"]+\.js)"', kissimy.text)[:3]
for js in js_files:
    js_url = f"https://storal.fr{js}"
    js_test = requests.get(js_url, timeout=5)
    status = "✅" if js_test.status_code == 200 else "❌"
    print(f"{status} {js} -> {js_test.status_code}")

Attente du rebuild complet...

=== TEST FINAL ===
Page d'accueil: 200
CSS demandés: ['/_next/static/css/c98d9c18f51f4126.css']
  /_next/static/css/c98d9c18f51f4126.css -> 404
    ❌ 404

=== TEST KISSIMY ===
Page Kissimy: 200
✅ /_next/static/chunks/4bd1b696-e5d7c65570c947b7.js -> 200
❌ /_next/static/chunks/3794-0fa92e90b939f74c.js -> 404
✅ /_next/static/chunks/main-app-c21d74f9618a768c.js -> 200


## Test du nouveau déploiement

Attente de 5 minutes pour le déploiement complet avec le nouveau script...

In [ ]:
# Attendre 5 minutes pour le déploiement complet
print("⏳ Attente du déploiement (5 minutes)...")
time.sleep(300)

print("\n🔍 VÉRIFICATION FINALE DU DÉPLOIEMENT")
print("=" * 50)

# Test page d'accueil
home = requests.get('https://storal.fr/', timeout=10)
print(f"\n📄 Page d'accueil: {home.status_code}")
css_home = re.findall(r'href="(/_next/static/css/[^"]+)"', home.text)
print(f"   CSS: {css_home[0] if css_home else 'Aucun'}")

if css_home:
    css_test = requests.get(f"https://storal.fr{css_home[0]}", timeout=5)
    if css_test.status_code == 200:
        print(f"   ✅ CSS accessible")
    else:
        print(f"   ❌ CSS 404")

# Test page Kissimy
kissimy = requests.get('https://storal.fr/products/kissimy', timeout=10)
print(f"\n📄 Page Kissimy: {kissimy.status_code}")
js_files = re.findall(r'src="(/_next/static/chunks/[^"]+\.js)"', kissimy.text)
js_ok = 0
js_ko = 0
for js in js_files[:5]:
    js_test = requests.get(f"https://storal.fr{js}", timeout=5)
    if js_test.status_code == 200:
        js_ok += 1
    else:
        js_ko += 1
        print(f"   ❌ {js} -> 404")
        
print(f"   JS files: {js_ok} ✅ / {js_ko} ❌")

# Test formulaire
if 'configurateur' in kissimy.text.lower():
    print(f"   ✅ Formulaire présent")
else:
    print(f"   ❌ Formulaire manquant")

print("\n" + "=" * 50)
if css_test.status_code == 200 and js_ko == 0:
    print("✅ DÉPLOIEMENT RÉUSSI!")
else:
    print("⚠️ Problèmes détectés")

## ✅ Vérification GitHub Actions Activé

Le workflow automatique est maintenant configuré ! Chaque push sur `main` déclenchera automatiquement le déploiement.

In [ ]:
# Vérifier le statut de GitHub Actions via l'API
import json

# Récupérer les workflows récents
api_url = "https://api.github.com/repos/jnabil1974/storal.fr/actions/runs"
try:
    response = requests.get(api_url, timeout=10)
    if response.status_code == 200:
        data = response.json()
        if 'workflow_runs' in data and len(data['workflow_runs']) > 0:
            latest = data['workflow_runs'][0]
            print("🔍 Dernier Workflow GitHub Actions:")
            print(f"   Status: {latest['status']}")
            print(f"   Conclusion: {latest.get('conclusion', 'En cours...')}")
            print(f"   Nom: {latest['name']}")
            print(f"   Démarré: {latest['created_at']}")
            print(f"   URL: {latest['html_url']}")
            
            if latest['status'] == 'completed':
                if latest['conclusion'] == 'success':
                    print("\n✅ Déploiement automatique réussi!")
                else:
                    print(f"\n❌ Déploiement échoué: {latest['conclusion']}")
            else:
                print("\n⏳ Déploiement en cours...")
        else:
            print("Aucun workflow trouvé. Vérifiez sur GitHub.")
    else:
        print(f"⚠️ Impossible d'accéder à l'API GitHub (status {response.status_code})")
except Exception as e:
    print(f"❌ Erreur: {e}")